# UDEA AI 4 – Modelo rápido con XGBoost (sin CV pesado)

Tercer enfoque: **XGBoost multiclase**, optimizado para que la ejecución sea razonablemente rápida en Kaggle/Colab.

- Usa `train.csv` y `test.csv` en el directorio de trabajo.
- Preprocesa categóricas con `OrdinalEncoder`.
- Hace un **solo split train/valid** (no cross-validation) para ahorrar tiempo.
- Entrena un modelo XGBoost ligero.
- Genera `submission_xgb_fast.csv` listo para subir a Kaggle.

## 0. (Opcional) Instalación de XGBoost si hiciera falta

In [ ]:
# En Kaggle y la mayoría de entornos XGBoost ya viene instalado.
# Si llegara a fallar el import, descomenta la siguiente línea:
# !pip install xgboost -q

## 1. Imports y configuración

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 2. Carga de datos

In [ ]:
!ls

train_path = 'train.csv'
test_path = 'test.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

print('Train shape:', train.shape)
print('Test shape:', test.shape)
train.head()

## 3. Eliminación de clases extremadamente raras

In [ ]:
vc = train['RENDIMIENTO_GLOBAL'].value_counts()
print(vc)

# Eliminamos clases con menos de 5 ejemplos para evitar problemas de estabilidad
rare = vc[vc < 5].index
print('\nEtiquetas raras a eliminar:', rare.tolist())

train = train[~train['RENDIMIENTO_GLOBAL'].isin(rare)].reset_index(drop=True)
print('Nuevo tamaño de train:', train.shape)

## 4. Preprocesamiento (OrdinalEncoder para categóricas)

In [ ]:
TARGET = 'RENDIMIENTO_GLOBAL'
ID_COL = 'ID'

y = train[TARGET]
X = train.drop(columns=[TARGET, ID_COL])
test_X = test.drop(columns=[ID_COL])

cat_cols = X.select_dtypes(include=['object']).columns.tolist()
num_cols = X.select_dtypes(exclude=['object']).columns.tolist()
print('Categóricas:', cat_cols)
print('Numéricas:', num_cols)

# Rellenamos nulos en categóricas
for col in cat_cols:
    X[col] = X[col].fillna('MISSING')
    test_X[col] = test_X[col].fillna('MISSING')

# OrdinalEncoder para convertir categóricas a números
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X[cat_cols] = enc.fit_transform(X[cat_cols])
test_X[cat_cols] = enc.transform(test_X[cat_cols])

# Codificamos la variable objetivo
le = LabelEncoder()
y_enc = le.fit_transform(y)
print('Clases:', le.classes_)

## 5. Split train / valid (sin cross-validation para ir más rápido)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y_enc,
    test_size=0.2,
    stratify=y_enc,
    random_state=RANDOM_STATE
)

X_train.shape, X_val.shape

## 6. Entrenamiento de XGBoost (modelo compacto para no tardar demasiado)

In [ ]:
xgb_params = {
    'n_estimators': 300,       # número moderado de árboles
    'max_depth': 6,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'tree_method': 'hist',     # más rápido en CPU
    'random_state': RANDOM_STATE,
    'n_jobs': -1               # usa todos los cores disponibles
}

model = XGBClassifier(**xgb_params)
model.fit(X_train, y_train)

val_pred = model.predict(X_val)
val_acc = accuracy_score(y_val, val_pred)
print('Accuracy en validación (hold-out):', val_acc)

## 7. Entrenamiento final en todo el train

In [ ]:
final_model = XGBClassifier(**xgb_params)
final_model.fit(X, y_enc)

## 8. Predicción sobre test y generación de submission

In [ ]:
test_pred = final_model.predict(test_X)
test_labels = le.inverse_transform(test_pred)

submission = pd.DataFrame({
    'ID': test[ID_COL],
    'RENDIMIENTO_GLOBAL': test_labels
})

submission.to_csv('submission_xgb_fast.csv', index=False)
submission.head()